# 目的: Excelの扱い方を学ぶ

In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib

In [109]:
df = pd.read_excel("../data/202307_imbalance-price_01.xlsx", sheet_name=None, header=None)

In [110]:
sheet_name = []
sheet_dfs = []
for k, i in df.items():
    sheet_name.append(k)
    sheet_dfs.append(i)

In [111]:
sheet_name

['Sheet1', '202307_imbalance-price_01', 'Sheet2', 'Sheet3']

In [112]:
def add_sheet_and_datablock_info(dfs, sheet_names):
    for i, name in enumerate(sheet_names):
        dfs[i]['sheet_name'] = name
        num_null_cols = dfs[i].isnull().sum(axis=1)
        num_filled_cols = (~dfs[i].isnull()).sum(axis=1)
        dfs[i]['diff_filled_cols'] = num_filled_cols - num_filled_cols.shift().fillna(0)
    return dfs

In [113]:
tmp_dfs = add_sheet_and_datablock_info(sheet_dfs, sheet_name)

In [114]:
tmp = tmp_dfs[2]

In [119]:
common_cols = ['sheet_name', 'diff_filled_cols']

In [115]:
def get_datablock_index(df, target_col, threshold):
    start_indexs = list(df[df[target_col]>=threshold].index.values)
    end_indexs = list(df[df[target_col]<=-1*threshold].index.values)
    start_indexs.append(df.index[0])
    end_indexs.append(df.index[-1])

    block_indexs = set(start_indexs + end_indexs)

    block_indexs = list(block_indexs)
    block_indexs.sort()
    return block_indexs

In [116]:
def make_datablock_ind_map(block_indexs):
    return_map = {}
    for i, start_index in enumerate(block_indexs):
        if (i + 2) == len(block_indexs):
            return_map[start_index] = 'End'
            return return_map
        else:
            end_index = block_indexs[i + 1]
            return_map[start_index] = end_index

In [117]:
block_indexs = get_datablock_index(tmp, 'diff_filled_cols', 5)

In [118]:
index_map = make_datablock_ind_map(block_indexs)

In [123]:
def get_common_col_map(df, common_cols):
    df_cols = df.columns
    common_cols_index, = np.where([(col in common_cols) for col in df_cols])
    common_col_map = {i:df_cols[i] for i in common_cols_index}
    return common_col_map

In [128]:
def rename_2_common_col(columns, common_col_map):
    for i, col in common_col_map.items():
        columns[i] = col
    return columns

In [129]:
def get_separated_datablock_list(df, block_indexs, common_cols):
    return_list = []
    common_col_map = get_common_col_map(df, common_cols)
    for s, e in block_indexs.items():
        if e=='End':
            data = df.iloc[(s+1):, :].values
            columns = df.iloc[s, :].values
        else:
            data = df.iloc[(s+1):e, :].values
            columns = df.iloc[s, :].values
        
        columns = rename_2_common_col(columns, common_col_map)
        separeted_df = pd.DataFrame(data=data, columns=columns)
        return_list.append(separeted_df)

    return return_list

In [130]:
test = get_separated_datablock_list(tmp, index_map, common_cols)

In [135]:
test = test[1]

In [143]:
test.head()

,行ラベル,20230701,20230702,20230703,20230704.0,20230705,20230706.0,20230707.0,20230708.0,20230709.0,...,20230716.0,20230717.0,20230718.0,20230719.0,20230720.0,20230721.0,20230722.0,総計,sheet_name,diff_filled_cols
0,1,11.1,10.15,10.38,9.75,13.35,12.09,11.15,14.13,12.81,...,10.39,10.91,13.72,15.89,11.24,10.96,9.19,255.38,Sheet2,0.0
1,2,9.79,10.38,11.96,8.84,10.46,11.37,10.14,13.83,12.22,...,10.05,11.64,12.78,11.92,11.73,10.2,9.42,241.11,Sheet2,0.0
2,3,9.8,9.98,10.73,7.84,8.27,10.46,10.04,13.2,21.49,...,14.95,10.5,12.75,12.45,11.41,10.92,9.25,250.96,Sheet2,0.0
3,4,9.3,10.19,10.3,7.84,9.7,11.2,9.93,13.24,12.59,...,12.16,10.0,11.86,12.17,11.92,9.93,10.05,234.84,Sheet2,0.0
4,5,11.08,10.21,11.59,11.5,8.84,10.9,10.05,12.96,12.36,...,13.08,8.93,10.5,12.21,11.91,9.8,9.34,236.49,Sheet2,0.0


In [140]:
non_time_cols = ['行ラベル', 'sheet_name', 'diff_filled_cols', '総計']
time_cols = [col for col in test.columns if col not in non_time_cols]

In [145]:
test = test.set_index('行ラベル')

In [148]:
test[time_cols].stack().reset_index()

,行ラベル,level_1,0
0,1,20230701.0,11.1
1,1,20230702.0,10.15
2,1,20230703.0,10.38
3,1,20230704.0,9.75
4,1,20230705.0,13.35
...,...,...,...
1061,49,20230717.0,50.641818
1062,49,20230718.0,66.214912
1063,49,20230719.0,59.271127
1064,49,20230720.0,41.247107


## 次